**<h1>IMPORT LIBRARIES</h1>**

In [1]:
import re
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.metrics import ndcg_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

**<h1>DATA PREPARATION</h1>**

In [2]:
file_path = './letterboxd_indo_movies_vectorized (bert-base-uncased).pkl'
with open(file_path, 'rb') as file:
    document = pickle.load(file)
document

,Title,Synopsis,Cast,Director,Producers,Writers,Modified Genres,director_vectorized,combined_vectorized,modified_genres_vectorized,synopsis_vectorized,title_vectorized,cast_vectorized,writer_vectorized,producer_vectorized
0,Whats Up with Cinta?,A popular high school girl strains her relatio...,"Dian Sastrowardoyo, Nicholas Saputra, Ladya Ch...",Rudi Soedjarwo,"Riri Riza, Mira Lesmana","Riri Riza, Mira Lesmana, Rako Prijanto, Jujur ...","romance, comedy, teens, drama","[[-0.6854871, -0.3914409, -0.5815962, 0.679251...","[[-0.76008993, -0.5792775, -0.982581, 0.684993...","[[-0.9194438, -0.63085705, -0.9925114, 0.91984...","[[-0.6546244, -0.50642717, -0.87920755, 0.6082...","[[-0.9298181, -0.5737379, -0.97136325, 0.87768...","[[-0.919351, -0.70150125, -0.9905906, 0.886945...","[[-0.79114574, -0.42002788, -0.98039776, 0.816...","[[-0.8825278, -0.42932817, -0.9291119, 0.86434..."
1,Whats Up with Cinta 2,14 years after their budding romance in high s...,"Dian Sastrowardoyo, Nicholas Saputra, Adinia W...",Riri Riza,"Mira Lesmana, Mandy Marahimin","Mira Lesmana, Prima Rusdi","romance, comedy, family, drama","[[-0.9085559, -0.47328997, -0.6515243, 0.81946...","[[-0.6102178, -0.49886593, -0.957828, 0.490051...","[[-0.9036535, -0.5764792, -0.9865643, 0.896646...","[[-0.51276034, -0.4523036, -0.9394405, 0.69975...","[[-0.8743504, -0.34147608, -0.57791084, 0.7501...","[[-0.9317873, -0.68486917, -0.9958121, 0.91637...","[[-0.91775936, -0.45918256, -0.9486102, 0.8836...","[[-0.8577517, -0.5698639, -0.968322, 0.8322123..."
2,Marlina the Murderer in Four Acts,After encountering a group of bandits with pla...,"Marsha Timothy, Egy Fedly, Tumpal Tampubolon, ...",Mouly Surya,"Fauzan Zidni, Rama Adi","Mouly Surya, Rama Adi","action, crime, gory, thriller, western, drama,...","[[-0.81094503, -0.35891283, -0.27210283, 0.691...","[[-0.7822325, -0.62164634, -0.9908777, 0.77250...","[[-0.83963627, -0.69027275, -0.9897743, 0.9077...","[[-0.8449, -0.477412, -0.89138496, 0.7166954, ...","[[-0.8915281, -0.4564107, -0.75647914, 0.80692...","[[-0.93924165, -0.7001719, -0.987454, 0.910688...","[[-0.74434674, -0.39309293, -0.82636666, 0.758...","[[-0.8608061, -0.48491392, -0.8843078, 0.82879..."
3,Fiction.,When a sheltered young woman becomes enamored ...,"Ladya Cheryl, Donny Alamsyah, Kinaryosih, Inon...",Mouly Surya,"Rama Adi, Tia Hasibuan, Sapto Soetardjo","Joko Anwar, Mouly Surya","thriller, drama","[[-0.81094503, -0.35891283, -0.27210283, 0.691...","[[-0.63022697, -0.52913153, -0.9620033, 0.5383...","[[-0.89362174, -0.47423673, -0.8815239, 0.8397...","[[-0.7762525, -0.50513536, -0.9343746, 0.70602...","[[-0.9207574, -0.38874292, -0.69215256, 0.8331...","[[-0.83717316, -0.6484815, -0.9975187, 0.85421...","[[-0.85910434, -0.5492014, -0.98309875, 0.8496...","[[-0.76274997, -0.50288624, -0.95507413, 0.807..."
4,Tjoet Nja Dhien,"Set in 1896, ""Tjoet Nja Dhien"" celebrates one ...","Christine Hakim, Slamet Rahardjo, Piet Burnama...",Eros Djarot,"Handi Muljono, Eros Djarot, Alwin Abdullah, Al...",None,"history, action, drama","[[-0.84725344, -0.36621767, -0.26003826, 0.759...","[[-0.8171015, -0.56236213, -0.9794723, 0.68467...","[[-0.91467106, -0.6066228, -0.9644351, 0.86765...","[[-0.18432444, -0.32774684, -0.9339838, 0.2748...","[[-0.8769051, -0.4881131, -0.7351499, 0.828920...","[[-0.9265778, -0.69146675, -0.9947356, 0.90776...","[[-0.79046774, -0.20977958, 0.42289498, 0.5920...","[[-0.7840522, -0.5085491, -0.9843888, 0.803804..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Puspa Indah Taman Hati,Galih met Marlina in Jogja while in college an...,"Prilly Latuconsina, Yesaya Abraham, Dewi Gita,...",Monty Tiwa,"Chand Parwez Servia, Fiaz Servia, Sumarsono",None,"romance, drama","[[-0.69146734, -0.18178421, 0.55461836, 0.3770...","[[-0.6774315, -0.5270221, -0.9794258, 0.597692...","[[-0.89238423, -0.44984433, -0.9418296, 0.8856...","[[-0.28373718, -0.3462334, -0.9417214, 0.36374...","[[-0.8738502, -0.27754354, -0.54477334, 0.7251...","[[-0.9

In [ ]:
def clean_string(s):
    s = s.replace('[', '').replace(']', '')
    s = s.replace('/', '')
    s = re.sub('[^\w\s\d,]', '', s)
    if s == '':
        return ''
    return s

In [ ]:
def list_to_clean_string(lst):
    combined_string = ' '.join(map(str, lst))
    s = re.sub(r'[\[\]/]', '', combined_string)
    return s

In [ ]:
document['Modified Genres'] = [clean_string(s) for s in document['Modified Genres']]

In [ ]:
document['Combined'] = document.apply(lambda row: [row['Title'], row['Synopsis'], row['Cast'], row['Director'], row['Producers'], row['Writers'], row['Modified Genres']], axis=1)

In [ ]:
document['Combined'] = [list_to_clean_string(s) for s in document['Combined']]

In [ ]:
movie_title = document['Title'].tolist()
movie_synopsis = document['Synopsis'].tolist()
movie_cast = document['Cast'].tolist()
movie_director = document['Director'].tolist()
movie_producers = document['Producers'].tolist()
movie_writers = document['Writers'].tolist()
movie_modified_genres = document['Modified Genres'].tolist()
movie_combined = [
    f"{row['Title']} {row['Synopsis']} {row['Cast']} {row['Director']} {row['Producers']} {row['Writers']} {row['Modified Genres']}"
    for _, row in document.iterrows()
]

**<h1>ENCODING DATASET</h1>**

In [3]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
def encode_text(text):
    inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.pooler_output.numpy()

In [ ]:
document['modified_genres_vectorized'] = document['Modified Genres'].apply(lambda x: encode_text(x))

In [ ]:
document['synopsis_vectorized'] = document['Synopsis'].apply(lambda x: encode_text(x))

In [ ]:
document['title_vectorized'] = document['Title'].apply(lambda x: encode_text(x))

In [ ]:
document['cast_vectorized'] = document['Cast'].apply(lambda x: encode_text(x))

In [ ]:
document['director_vectorized'] = document['Director'].apply(lambda x: encode_text(x))

In [ ]:
document['writer_vectorized'] = document['Writers'].apply(lambda x: encode_text(x))

In [ ]:
document['producer_vectorized'] = document['Producers'].apply(lambda x: encode_text(x))

In [ ]:
document['combined_vectorized'] = document['Combined'].apply(lambda x: encode_text(x))

**<h1>MOVIE RECOMMENDATION</h1>**

In [5]:
synopsis_vectors = np.vstack(document['synopsis_vectorized'].values)

In [6]:
cast_vectors = np.vstack(document['cast_vectorized'].values)

In [7]:
director_vectors = np.vstack(document['director_vectorized'].values)

In [8]:
producer_vectors = np.vstack(document['producer_vectorized'].values)

In [9]:
writer_vectors = np.vstack(document['writer_vectorized'].values)

In [10]:
title_vectors = np.vstack(document['title_vectorized'].values)

In [11]:
modified_genres_vectors = np.vstack(document['modified_genres_vectorized'].values)

In [12]:
combined_vectors = np.vstack(document['combined_vectorized'].values)

In [13]:
def semantic_search(input_keywords, df, top_k=5):
    results = []

    valid_query_types = {
        'synopsis': synopsis_vectors,
        'cast': cast_vectors,
        'director': director_vectors,
        'producer': producer_vectors,
        'writer': writer_vectors,
        'title': title_vectors,
        'genres': modified_genres_vectors
    }

    for query_type, query in input_keywords.items():
        if query_type not in valid_query_types:
            raise ValueError(f'Invalid query type: {query_type}. Valid types are: {list(valid_query_types.keys())}')

        vectors = valid_query_types[query_type]

        query_vector = encode_text(query).reshape(1, -1)
        similarities = cosine_similarity(query_vector, vectors).flatten()
        top_k_indices = similarities.argsort()[-top_k:][::-1]
        top_k_results = df.iloc[top_k_indices]
        top_k_similarities = similarities[top_k_indices]

        results.append((top_k_results, top_k_similarities))

    return results

In [14]:
input_keywords = {
    'genres': 'horror'
}
target_genre = input_keywords['genres']

search_results = semantic_search(input_keywords, document)
predicted_similarities = []
actual_relevance = []

for result in search_results:
    top_k_results, top_k_similarities = result

    predicted_similarities.extend(top_k_similarities)

    actual_relevance.extend([1 if target_genre.lower() in genres.lower() else 0 for genres in top_k_results['Modified Genres']])

    for idx, (index, res, score) in enumerate(zip(top_k_results.index, top_k_results[['Title', 'Modified Genres']].values, top_k_similarities)):
        print(f'Result {idx + 1}: [{index}] {res[0]} is a {res[1]} movie, Confidence Score: {score}')

Result 1: [50] Keramat is a horror movie, Confidence Score: 0.9999999403953552
Result 2: [144] Sacred 2 is a horror movie, Confidence Score: 0.9999999403953552
Result 3: [143] Sacred is a horror movie, Confidence Score: 0.9999999403953552
Result 4: [56] Ghost with Hole is a horror movie, Confidence Score: 0.9999999403953552
Result 5: [149] Innocent Vengeance is a thriller movie, Confidence Score: 0.8521703481674194


**<h1>EVALUATE MODEL</h1>**

In [15]:
actual_relevance = np.array([actual_relevance])
predicted_similarities = np.array([predicted_similarities])

In [16]:
actual_relevance_1d = actual_relevance.flatten()
predicted_similarities_1d = predicted_similarities.flatten()

<h3>Mean Absolute Error</h3>

In [17]:
mae = mean_absolute_error(actual_relevance_1d, predicted_similarities_1d)
print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 0.1704341173171997


<h3>Root Mean Squared Error</h3>

In [18]:
rmse = np.sqrt(mean_squared_error(actual_relevance_1d, predicted_similarities_1d))
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 0.3811021653824064


<h3>Normalized Discounted Cumulative Gain</h3>

In [19]:
ndcg = ndcg_score(actual_relevance, predicted_similarities, k=len(predicted_similarities[0]))
print(f'NDCG Score: {ndcg}')

NDCG Score: 1.0
